# Search Area
- The search area should be a simple sqauare grid (2-dimentional array)
- Each element in the 2-dimentional array represents a square/node 
- There is a start node and an end node (square)
- Each node is either considered walkable or not walkable

In [1]:
# Need to create a node class
'''
Each node contains the following attributes:
- a position on the board. Tuple like (1,1) i.e. second row 
second column element.
- parent node position
- type of node (walkable or unwalkable) boolean
- G-Cost
- H-Cost
- F-Score
'''
class Node:
    '''Node class for algo'''
    
    def __init__(self, x, y, walkable):
        self.x = x
        self.y = y
        self.walkable = walkable
        self.position = None
        self.parent = None
        self.g = 0 
        self.h = 0 
        self.f = 0
      
    '''Creating methods to populate node data and attributes'''
    
    def set_position(self): # sets the nodes position (identity) 
        self.position = (self.y, self.x)
        
    def g_cost(self): # calculates g_cost and sets it to node g attribute
        distance = ((self.x - self.parent.x)**2 + (self.y - self.parent.y)**2)**0.5
        if distance == 1:
            self.g += (self.parent.g + 10)
        else:
            self.g += (self.parent.g + 14)
    
    def h_cost(self, end_node): # calculates h_cost and sets it to node h attribute
        self.h = (abs(self.x - end_node.x) + abs(self.y - end_node.y)) * 10
    
    def f_score(self): # calculates f_score and sets it to node f attribute
        self.f = self.g + self.h
        
        


In [2]:
import math

In [3]:
# The skeleton will be used to display the path traveled
# this will also be used to develope a map of nodes 
# zeros mean "walkable" nodes whereas ones mean "not walkable"

def node_map(skeleton):
    '''
    This function will create a map of nodes based off
    of a two dimentional array of ones and zeros.
    '''
    node_map = [[0 for _ in range(len(skeleton[0]))] for _ in range(len(skeleton))]
    
    for y in range(len(skeleton)):
        for x in range(len(skeleton[0])):
            skeleton_item = skeleton[y][x]
            if skeleton_item == 1:
                node = Node(x, y, False)
                node.set_position()
                node_map[y][x] = node
            else:
                node = Node(x, y, True)
                node.set_position()
                node_map[y][x] = node
    
    return node_map




# Starting The Search 
- What we need to to is find the most efficient path from the start node to the end node
- Put simply we start at the start node check the adjeacent squares then keep searching outward from the start node until we find the end node
## Starting The Search
    1.) we need an open list and closed list
    - Open list will contain the all of the nodes that need to be checked out
    - Closed list contains all the squares that have already been checked out and need no further examination
    2.) Add the start node to the open list
    
    3.) Find all the squares surrounding the start node (adjacent squares)
    - Add these adjacent nodes to the open list for further examination.
    - For each adjacent square/node make the start node their "Parent Node".
    4.) Finally drop the start node from the open list and add it to the closed list.
   

In [4]:
def find_open_elements(arr, row, col, open_list, closed_list):
    """
    Find all the elements surrounding a given element in a 2D array,
    including diagonal positions.

    Args:
        arr (list[list]): The 2D array.
        row (int): The row index of the element.
        col (int): The column index of the element.

    Returns:
        list: A list of elements surrounding the given element.
    """
    # Define the possible directions: top, bottom, left, right,
    # top-left, top-right, bottom-left, bottom-right
    directions = [(0, -1), (-1, 0), (0, 1), (1, 0),
                  (-1, -1), (-1, 1), (1, -1), (1, 1)]
    new_nodes = []

    # Iterate through each direction
    for direction in directions:
        new_row = row + direction[0]
        new_col = col + direction[1]

        # Check if the new row and column are within the array bounds
        if 0 <= new_row < len(arr) and 0 <= new_col < len(arr[0]):
            # If so, append the element at the new coordinates to the result list
            if (arr[new_row][new_col].walkable == True) and (arr[new_row][new_col] not in open_list and (arr[new_row][new_col] not in closed_list)):
                arr[new_row][new_col].parent = arr[row][col]
                new_nodes.append(arr[new_row][new_col])

    return new_nodes

    

In [5]:
def kick_start(node_map, skeleton):
    "Function for finding the optimal path"
    
    #create an open list
    open_list = []
    #create a closed list
    closed_list = []
    
    # initialize a start node
    start_node = node_map[0][0]
    # Initialize an end node
    end_node = node_map[-1][-1]
    
    # insert the start node into the open list
    open_list.append(start_node)
    
    current_node = open_list.pop()
    row = current_node.y
    column = current_node.x
    
    # Find the nodes surrounding the start nodes
    new_nodes = find_open_elements(node_map, row, column, open_list, closed_list)
    open_list.extend(new_nodes)
    
    # insert the current_node (start_node) into the closed list
    closed_list.append(current_node)
    
        

    return open_list, closed_list, end_node
    

    
    

In [6]:
def FindPath(open_list, closed_list, end_node, node_map):
    # As long as the end node is not in the closed list do the following:

    while end_node not in closed_list:
        # calculate the g, h, and f scores for each of the nodes in the open list
        for item in open_list:
            item.g_cost()
            item.h_cost(end_node)
            item.f_score()

        # sort the open list based on the lowest F-Score
        open_list.sort(key=lambda x: x.f, reverse=False)

        # Initialize a current node, The first one in open list will be sufficient
        current_node = open_list.pop(0)
        row = current_node.y
        column = current_node.x
        # find all the new nodes surrounding the current node
        # make these nodes parent node the current node
        new_nodes = find_open_elements(node_map, row, column, open_list, closed_list)

        # test whether the nodes already in the open list have a lower g-cost
        # If they do benefit from this new path make the parent of that node the current node

        #extend the open list
        open_list.extend(new_nodes)
        closed_list.append(current_node)
    
    return True, closed_list.pop()
    
    
    
    

In [7]:
def paint_path(skeleton, end_node):
    for row in skeleton:    
        print(row)
    print("\n")
    skeleton[end_node.position[0]][end_node.position[-1]] = 2
    current_node = end_node.parent
    while True:
        skeleton[current_node.position[0]][current_node.position[-1]] = 2
        current_node = current_node.parent
        if current_node.parent == None:
            skeleton[current_node.position[0]][current_node.position[-1]] = 2
            break
      
    for row in skeleton:
        print(row)
    

In [8]:
def main():
    skeleton = [
    [0,0,0,0],
    [0,1,1,0],
    [0,1,1,0],
    [0,1,1,0]
]
    
    mapper = node_map(skeleton)
    open_list, closed_list, end_node = kick_start(mapper, skeleton)
    path, end_node = FindPath(open_list, closed_list, end_node, mapper)
    if path == True:
        paint_path(skeleton, end_node)
    
    

In [9]:

main()


[0, 0, 0, 0]
[0, 1, 1, 0]
[0, 1, 1, 0]
[0, 1, 1, 0]


[2, 2, 2, 0]
[0, 1, 1, 2]
[0, 1, 1, 2]
[0, 1, 1, 2]


# Path Scoring 
- The next procedure is to determine which node we should travel to after the start node.
- The possible nodes that we can travel to are nodes that are "walkable" and dont act as walls that are in the open list.
## Path Scoring
- The node we choose to travel to is the one with the smallest F Score
    
    1.) F-Score = G-Cost + Hueristic (F = G + H)
    
    - G-Cost is the cost of moving from the start node to any other square/node on the board.
        - Cost of 10 to move horozontally or vertically one node, and a cost of 14 to move diagonally to a node.
    - To calculate g-cost of a node add 10 or 14 to the parent nodes g-cost.
    
    
    - H-Cost us the estimated movement cost from the current square to the final destination (end node) also it is important to note that this is an estimation.
        - Calculating H by the Manhattan method. The number of horzontal and ventical nodes from a specific node to an end node multiplied by 10.
    
    
    - F-score is calculated by adding the previous costs together
    
"Our path is generated by repeatedly going through our open list and choosing the square with the lowest F score."
  

#  Continuing the search 
    1.) Choose the node with lowest F-Score in the open list
       - Drop this specific node from the open list and add it to the closed list.
       
       
    2.) Now identify all of the adjacent squares and add them to the open list if they are not already in the open list, not in the closed list, and not unwalkable.
    - For the newly added squares to the open list make the current square the parent square of these nodes.
    
    
    3.) If an adjacent node has already been on the open list...
    - check to see if the G score for that square is lower if we use the current square to get there. If not, don’t do anything.
    - If the G-Cost of the new path is lower make the current node the parent node for that node
    
    
    4.) Sort open list from lowest to highest f-scores (and choose lowest f-score. 
    
    
Keep repeating this above process until we have found the end node and add it to our closed list

